In [59]:
training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

header = ["color", "diameter", "label"]

In [60]:
def unique_values(rows, cols):
    return set([row(col) for row in rows])

In [61]:
def class_counts(rows):
   # """"Counts the number of each type of example in a dataset.""""
    counts={} # a dictionary of label -> count
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts
        


In [62]:
def is_numeric(value):
    # test if value is numeric
    return isinstance(value, int) or isinstance(value, float)


In [63]:
class Question:
    # A question is used to partitian the dataset
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        # compare the feature value in an example
        # to the feature value in this question
        val = example[self.column]
        if is_numeric(val):
            return val>= self.value
    
    def __repr__(self):
        # helper method to print the question
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
        header[self.column], condition, str(self.value)
        )
    

In [64]:
def partition(rows, question):
    # partitions a dataset
    # for each row in the dataset check if it matches the question
    # If so, add it to "true rows", otherwise add it to "false rows"
    
    true_rows, false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [65]:
def gini(rows):
    # calc the gini impurity for a list of rows
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [66]:
def info_gain(left, right, current_uncertainty):
    # the uncertainty of the the starter node minus the weighted impurity
    # of the child nodes
    p= float(len(left)) /(len(left)+len(right))
    return current_uncertainty - p*gini(left) - (1-p)*gini(right)

In [67]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


In [68]:
class Leaf:
    # A leaf node classifies data
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    # a decision node asks a question
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [69]:
def build_tree(rows):
    # try partitioning the dataset on each unique attribute
    # calc the information gain
    # and return the question that produces the highest information gain.
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
    return
    
    # print the qestion at this node
    print(spacing + str(node.question))
    
    print(spacing + "-> True:")
    print_tree(node.true_branch, spacing + " ")
    
    print(spacing + "-> False:")
    print_tree(node.false_branch, spacing + " ")

In [73]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


In [74]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs


In [75]:
if __name__ == '__main__':

    my_tree = build_tree(training_data)

    print_tree(my_tree)

    # Evaluate
    testing_data = [
        ['Green', 3, 'Apple'],
        ['Yellow', 4, 'Apple'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]

    for row in testing_data:
        print ("Actual: %s. Predicted: %s" %
               (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Mango': '66%', 'Lemon': '33%'}
Actual: Apple. Predicted: {'Mango': '66%', 'Lemon': '33%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '66%', 'Lemon': '33%'}
